# 📓 Notebook Manager

This cell initializes the widgets required for managing your research notebook. Please run the cell below to enable functionality for:
- Exporting cells tagged with `export` into a `clean` notebook
- Generating a dynamic Table of Contents (TOC)
- Exporting the notebook to GitHub-compatible Markdown

➡️ **Be sure to execute the next cell before continuing with any editing or exporting.**

In [5]:
# Cell 1 - Workflow Tools
import sys
sys.path.insert(0, '../../lib')

from notebook_tools import TOCWidget, ExportWidget
import ipywidgets as widgets


# Create widget instances
toc = TOCWidget()
export = ExportWidget()

# Create horizontal layout
left_side = widgets.VBox([toc.button, export.button, toc.status])
right_side = widgets.VBox([toc.output, export.output])

# Display side by side
display(widgets.HBox([left_side, right_side]))

# 📑 Table of Contents (Auto-Generated)

This section will automatically generate a table of contents for your research notebook once you run the **Generate TOC** function. The table of contents will help you navigate through your data collection, analysis, and findings as your citizen science project develops.

➡️ **Do not edit this cell manually. It will be overwritten automatically.**


<!-- TOC -->
# Table of Contents


<!-- /TOC -->


## 🔧 Environment Setup

This cell establishes the batch preprocessing environment by:

1. **Importing Required Libraries**
  - OpenCV (cv2) for video processing and frame extraction
  - NumPy for array operations
  - Pandas for organizing metadata and results
  - Pathlib for cross-platform file path handling
  - JSON for checkpoint persistence
  - Datetime for timestamp parsing and filtering
  - Logging for process tracking

2. **Setting System Paths**
  - Adding mlops_ops modules to Python path
  - Verifying access to preprocessing utilities

3. **Initializing Checkpoint System**
  - Loading any previous processing state
  - Setting up progress tracking variables
  - Establishing failure recovery mechanism

**Note**: Run this cell first to ensure all dependencies are available before proceeding with batch processing.

In [10]:
# Cell 2 - Environment Setup
import numpy as np
import pandas as pd
from pathlib import Path
import json
from datetime import datetime, timedelta
import logging
import os
import sys

# Add mlops modules to path
sys.path.insert(0, '../lib')

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Check for OpenCV
try:
    import cv2
    print(f"✓ OpenCV version: {cv2.__version__}")
except ImportError:
    print("⚠️ OpenCV not installed. Install with: pip install opencv-python")
    print("   Continuing without video processing capabilities...")
    cv2 = None

print(f"✓ Python version: {sys.version.split()[0]}")
print(f"✓ Working directory: {os.getcwd()}")

⚠️ OpenCV not installed. Install with: pip install opencv-python
   Continuing without video processing capabilities...
✓ Python version: 3.12.9
✓ Working directory: /home/trauco/v3-traffic-vision/notebooks/MLOps


## 📐 Batch Processing Configuration

Define the core parameters for daily batch preprocessing:

- **Target Time**: Extract frames from videos closest to 12:00 PM EST
- **Date Filter**: Process only videos from yesterday (full calendar day)
- **Frame Count**: Number of frames to extract per video
- **Input Path**: Base directory containing camera subdirectories
- **Output Path**: Where to save extracted frames
- **File Pattern**: Expected video filename format (CAMERA_YYYYMMDD_HHMMSS.mp4)

This configuration serves as the single source of truth for the batch processing workflow.

In [11]:
# Cell 3 - Batch Processing Configuration
from datetime import datetime, timedelta

# Configuration
CONFIG = {
   # Time targeting
   'TARGET_TIME': '120000',  # 12:00:00 in 24-hour format
   'TARGET_HOUR': 12,
   
   # Date filtering - yesterday only
   'PROCESS_DATE': (datetime.now() - timedelta(days=1)).strftime('%Y%m%d'),
   
   # Frame extraction
   'FRAMES_PER_VIDEO': 10,
   
   # Paths
   'INPUT_DIR': Path.home() / 'traffic-recordings',
   'OUTPUT_DIR': Path('batch_processed_frames'),
   
   # File pattern
   'VIDEO_PATTERN': '*_{date}_*.mp4',  # Will be formatted with PROCESS_DATE
   'FILENAME_FORMAT': '{camera}_{date}_{time}.mp4'  # Expected format
}

# Display configuration
print("Batch Processing Configuration:")
print(f"  Target Date: {CONFIG['PROCESS_DATE']}")
print(f"  Target Time: {CONFIG['TARGET_TIME']} (12:00:00)")
print(f"  Frames per video: {CONFIG['FRAMES_PER_VIDEO']}")
print(f"  Input: {CONFIG['INPUT_DIR']}")
print(f"  Output: {CONFIG['OUTPUT_DIR']}")

Batch Processing Configuration:
  Target Date: 20250620
  Target Time: 120000 (12:00:00)
  Frames per video: 10
  Input: /home/trauco/traffic-recordings
  Output: batch_processed_frames


## 💾 Initialize Checkpoint System

Create checkpoint functionality to track processing progress and enable recovery from interruptions. This system saves state after each video completes, allowing the workflow to resume from the last successful video if stopped.

In [12]:
# Cell 4 - Initialize Checkpoint System
CHECKPOINT_FILE = "batch_preprocessing_checkpoint.json"
start_time = datetime.now()

def load_checkpoint():
   """Load previous progress if exists"""
   if Path(CHECKPOINT_FILE).exists():
       with open(CHECKPOINT_FILE, 'r') as f:
           checkpoint = json.load(f)
           print(f"✓ Loaded checkpoint: {len(checkpoint['processed'])} videos already processed")
           return checkpoint
   return {
       "processed": [], 
       "failed": [], 
       "last_completed": None,
       "process_date": CONFIG['PROCESS_DATE'],
       "start_time": start_time.isoformat()
   }

def save_checkpoint(checkpoint):
   """Save current progress"""
   checkpoint['last_updated'] = datetime.now().isoformat()
   with open(CHECKPOINT_FILE, 'w') as f:
       json.dump(checkpoint, f, indent=2)

# Initialize checkpoint
checkpoint = load_checkpoint()

# Verify checkpoint is for current date
if checkpoint.get('process_date') != CONFIG['PROCESS_DATE']:
   print(f"⚠️  Checkpoint is from {checkpoint.get('process_date')}, starting fresh for {CONFIG['PROCESS_DATE']}")
   checkpoint = {
       "processed": [], 
       "failed": [], 
       "last_completed": None,
       "process_date": CONFIG['PROCESS_DATE'],
       "start_time": start_time.isoformat()
   }

print(f"✓ Checkpoint system ready")

✓ Checkpoint system ready


## 📂 Scan Video Directories

Enumerate all camera subdirectories and count available videos from yesterday's date. This provides an overview of the data available for processing and identifies any cameras that may be missing recordings.

In [14]:
# Cell 5 - Scan Video Directories
camera_dirs = sorted([d for d in CONFIG['INPUT_DIR'].iterdir() if d.is_dir() and d.name.startswith('ATL-')])
print(f"Found {len(camera_dirs)} camera directories\n")

# Count videos per camera for yesterday
video_counts = {}
pattern = CONFIG['VIDEO_PATTERN'].format(date=CONFIG['PROCESS_DATE'])

for cam_dir in camera_dirs:
   videos = list(cam_dir.glob(pattern))
   video_counts[cam_dir.name] = len(videos)
   
   if len(videos) == 0:
       print(f"⚠️  {cam_dir.name}: No videos from {CONFIG['PROCESS_DATE']}")
   else:
       print(f"✓ {cam_dir.name}: {len(videos)} videos")

total_videos = sum(video_counts.values())
print(f"\nTotal videos available: {total_videos}")
print(f"Cameras with recordings: {sum(1 for v in video_counts.values() if v > 0)}/{len(camera_dirs)}")

Found 31 camera directories

⚠️  ATL-0006: No videos from 20250620
⚠️  ATL-0027: No videos from 20250620
⚠️  ATL-0069: No videos from 20250620
⚠️  ATL-0080: No videos from 20250620
⚠️  ATL-0150: No videos from 20250620
⚠️  ATL-0540: No videos from 20250620
⚠️  ATL-0610: No videos from 20250620
⚠️  ATL-0612: No videos from 20250620
⚠️  ATL-0613: No videos from 20250620
⚠️  ATL-0907: No videos from 20250620
⚠️  ATL-0917: No videos from 20250620
⚠️  ATL-0922: No videos from 20250620
⚠️  ATL-0943: No videos from 20250620
⚠️  ATL-0946: No videos from 20250620
⚠️  ATL-0947: No videos from 20250620
⚠️  ATL-0948: No videos from 20250620
⚠️  ATL-0952: No videos from 20250620
⚠️  ATL-0972: No videos from 20250620
⚠️  ATL-0973: No videos from 20250620
⚠️  ATL-0980: No videos from 20250620
⚠️  ATL-0981: No videos from 20250620
⚠️  ATL-0987: No videos from 20250620
⚠️  ATL-0992: No videos from 20250620
⚠️  ATL-0996: No videos from 20250620
⚠️  ATL-0997: No videos from 20250620
⚠️  ATL-0998: No vide

In [15]:
# Cell 5b - Debug: Check actual video files
# Check first camera directory for actual file pattern
test_cam = camera_dirs[0]
print(f"Checking {test_cam.name} for all video files:")

all_videos = list(test_cam.glob("*.mp4"))
print(f"Total videos: {len(all_videos)}")

if all_videos:
    # Show first 5 filenames to see pattern
    print("\nSample filenames:")
    for video in all_videos[:5]:
        print(f"  {video.name}")
    
    # Check for yesterday's date in any position
    yesterday_videos = [v for v in all_videos if CONFIG['PROCESS_DATE'] in v.name]
    print(f"\nVideos containing {CONFIG['PROCESS_DATE']}: {len(yesterday_videos)}")

Checking ATL-0006 for all video files:
Total videos: 0
